# Download all twitter politicians

The following webpage has all the danish politician twitterprofiles  https://filip.journet.sdu.dk/twitter/politikere/

In [1]:
import requests
from bs4 import BeautifulSoup
import re #https://developers.google.com/edu/python/regular-expressions
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm
!pip install emoji
from emoji import UNICODE_EMOJI

In [2]:
from tqdm.notebook import tqdm
URL = 'https://filip.journet.sdu.dk/twitter/politikere/'
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')


## Selecting the twitternames

See that the twitter names is in a h3 style:

In [4]:
twitter_names = soup.find_all('h3')

In [5]:
#twitter_names

In [6]:
twitter_name_list=[]

In [7]:
for names in twitter_names:
    #print(names, end='\n'*2)
    match=re.search("<h3>#",str(names))
    if match:
        result_name=re.findall(".com/\w*",str(names))[0].replace(".com/","")
        twitter_name_list.append(result_name.rstrip())
        print(result_name)

vestager
larsloekke
oestergaard
PSkipperEL
IdaAuken
Kristian_Jensen
uffeelbaek
SorenPape
PiaOlsen
Heunicke
sofiecn
Statsmin
regeringDK
brianholm1962
ammitzbollbille
JakobEllemann
DanJoergensen
DanishMFA
mattiastesfaye
RosenkrantzT
EllenTraneNorby
MrMesserschmidt
FrankJensenKBH
sophieloehde
aahlers
martinlidegaard
karmel80
zeniastampe
Astridkrag
BennyEngelbrech
RasmusJarlov
PHummelgaard
JanEJoergensen
StemLAURITZEN
MogensJensenS
metteabildgaard
mfMorten
Kristianthdahl
RasmusPrehn
AlexVanopslagh
PerClausen3
jacobmark_sf
_MaiMercado
Jesper_Pet
skaarup_df
nvillumsen
Trinebramsen
PernilleVermund
SosseSass
MargreteAuken
rohde_jens
MetteGjerskov
LotteRod
mortenhelveg
UFM_MIN
espersendf
olebirkolesen
Pia_Kjaersgaard
RosaLundEl
LisbethBNielsen
rasmusnordqvist
SophieHAndersen
JeppeKofod
AneHalsboe
SchaldemoseMEP
Finansmin
Loekkegaard_MEP
engelschmidt
simonkollerup
KHegaard
khoenge
Jens_Joel
Erhvervs_min
DKsundhed
Torstenschack
Ulla_Tornaes
rasmushorn
michaelaastrup
KofodPeter
miljoemin
naserkhad

## The real names

The real names is in a < small > class.

In [8]:
real_name_list=[]

In [9]:
real_names = soup.find_all('small')

In [10]:
for r_names in real_names:

        
    result=re.search(r'<small>.+',str(r_names)).group(0).replace("<small>","")
    real_name_list.append(result.rstrip())
    if result.rstrip() in UNICODE_EMOJI:
        print('EMOJI')
        
    print(result)
    

Margrethe Vestager 
Lars Løkke Rasmussen 
Morten Østergaard 
Pernille Skipper 
Ida Auken 
Kristian Jensen 
Uffe Elbaek 
Søren Pape Poulsen 
Pia Olsen Dyhr 
Magnus Heunicke 
Sofie CarstenNielsen 
Statsministeriet 
Regeringen 
Brian Holm 
Simon Emil Ammitzbøll-Bille 
Jakob Ellemann-Jensen 
Dan Jørgensen 
Denmark MFA 🇩🇰 
Mattias Tesfaye 
Rosenkrantz-Theil 
Ellen Trane Nørby 
Morten Messerschmidt 
Frank Jensen 
Sophie Løhde 
Tommy Ahlers 
Martin Lidegaard 
Karen Melchior😷🧼🇪🇺 
Zenia Stampe 
Astrid Krag 
Benny Engelbrecht 
Rasmus Jarlov 
Peter Hummelgaard 
Jan E. Jørgensen 
Karsten Lauritzen 
Mogens Jensen 
Mette Abildgaard 
Morten Bødskov 
Kristian Thulesen Dahl 
Rasmus Prehn 
Alex Vanopslagh 
Per Clausen 
Jacob Mark 
Mai Mercado 
Jesper Petersen 
Peter Skaarup 
Nikolaj Villumsen 
Trine Bramsen 
Pernille Vermund 
Henrik Sass 
Margrete Auken 
Jens Rohde 
Mette Gjerskov 
Lotte Rod 
Morten Helveg 
Uddannelses- og Forskningsministeriet 
Søren Espersen 
Ole Birk Olesen 
Pia Kjærsgaard 
Rosa Lund

In [11]:
len(real_name_list)

651

## Making dataframe

In [12]:
d={'Names':real_name_list,"Twittername":twitter_name_list}
data=pd.DataFrame(data=d)

In [13]:
data.to_csv("danish_politkere_twitter.csv")

In [14]:
data.head()

,Names,Twittername
0,Margrethe Vestager,vestager
1,Lars Løkke Rasmussen,larsloekke
2,Morten Østergaard,oestergaard
3,Pernille Skipper,PSkipperEL
4,Ida Auken,IdaAuken


## Finding policial parties from https://www.danskepolitikere.dk/

In [15]:
# load csv
data = pd.read_csv("danish_politkere_twitter.csv")
data = data.drop(columns=['Unnamed: 0'])
soup = BeautifulSoup(requests.get('https://www.danskepolitikere.dk/oversigt/danske-politikere').content, 'html.parser')

In [16]:
sections = list(soup.find_all('section'))

In [17]:
# add "UNKNOWN" as default party
data['Party'] = 'UNKNOWN'
data['Region'] = 'UNKNOWN'
data.head()

,Names,Twittername,Party,Region
0,Margrethe Vestager,vestager,UNKNOWN,UNKNOWN
1,Lars Løkke Rasmussen,larsloekke,UNKNOWN,UNKNOWN
2,Morten Østergaard,oestergaard,UNKNOWN,UNKNOWN
3,Pernille Skipper,PSkipperEL,UNKNOWN,UNKNOWN
4,Ida Auken,IdaAuken,UNKNOWN,UNKNOWN


In [18]:
parties = []
# cross reference for our politicians and check if he's in the party
for row, coloumns in data.iterrows(): 
    name = coloumns[0] 
    for section in sections:
        party = section.h3.a.text # gets the party 
        if re.search(name.lower(), str(section).lower()):
            data.at[row, 'Party'] = party
            #print("Found " + name + " in " + party)
            
        


In [19]:
# view results
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,1,1,1
Dansk Folkeparti,13,13,13
Det Konservative Folkeparti,14,14,14
Enhedslisten,12,12,12
Inuit Ataqatigiit,1,1,1
Liberal Alliance,5,5,5
Radikale Venstre,15,15,15
Socialdemokratiet,42,42,42
Socialistisk Folkeparti,19,19,19


In [20]:
# get snapshot of who's missing
data.loc[data['Party'] == "UNKNOWN"]


,Names,Twittername,Party,Region
6,Uffe Elbaek,uffeelbaek,UNKNOWN,UNKNOWN
10,Sofie CarstenNielsen,sofiecn,UNKNOWN,UNKNOWN
12,Regeringen,regeringDK,UNKNOWN,UNKNOWN
13,Brian Holm,brianholm1962,UNKNOWN,UNKNOWN
14,Simon Emil Ammitzbøll-Bille,ammitzbollbille,UNKNOWN,UNKNOWN
...,...,...,...,...
646,Molle lykke nielsen,LykkeMolle,UNKNOWN,UNKNOWN
647,Vibeke Gamst,vibeke_gamst,UNKNOWN,UNKNOWN
648,Pia Moldt,piamoldt,UNKNOWN,UNKNOWN
649,Lars Folmann,larsfolm,UNKNOWN,UNKNOWN


Too many still unkown so we must look elsewhere, in denmark we have many
regions with politicans that aren't necsarily members of the parlament
![Regioner i dk](https://rn.dk/-/media/Rn_dk/Genveje/Fakta-om-Nordjylland/Regioner-i-Danmark/Danmark_Regioner_2017.ashx?la=da)
So we will scrape these websites as well


### first north jutland

In [21]:
nordjylland_url = "https://rn.dk/da/Politik/De-regionale-politikere"

soup = BeautifulSoup(requests.get(nordjylland_url).content, 'html.parser')



In [22]:
# examine html
#print(soup.prettify())
match_rnj = []
pre_re = r"\b[A-Z].*"
div_lsit = soup.findAll("ul", {'class': 'span3'})
for paragraph in div_lsit:
    text= paragraph.text
    text_stripped = text.rstrip().lstrip()
    #print(text_stripped)
    matches_1 = re.findall(pre_re, text_stripped)
    print(matches_1)
    match_rnj.append(matches_1)

# 0 and even indices are names, 1 and odd are the matching parti strings

['Budget og regnskab', 'Dine rettigheder når regionen behandler oplysninger om dig', 'Digital post', 'Indkøb', 'Licitationer og udbud']
['KlimaRegion', 'Organisation', 'Strategier og politikker', 'Samarbejde med erhvervsliv']
['Design', 'Sociale medier', 'Vision og retning', 'Sundhedstegn - podcast']
['De regionale politikere', 'Regionsrådet', 'Se Regionsrådsmøder', 'Dialog med politikerne i Region Nordjylland']
['Politiske udvalg under Regionsrådet', 'Forretningsudvalget', 'Kontaktudvalget', 'Sundhedskoordinationsudvalget']
['Vederlag for politisk arbejde', 'Partistøtte']
['Akut sygdom', 'Hospitaler']
['Dine rettigheder som patient', 'Praktiserende læger og behandlere']
['NemSMS']
['IV behandling i kommunalt regi', 'Forskning', 'Kontaktliste for kommunernes hjemmepleje', 'Kontaktliste til kommuner på børneområdet', 'National og Tværsektoriel IT', 'Nord-KAP']
['Samskabelse', 'Planer på sundhedsområdet', 'Samarbejde med kommuner omkring infektionshygiejne', 'Folkesundhed', 'Sundhedsafta

In [23]:
# note that relevant matches come at last 2 indices
good_match = match_rnj[len(match_rnj)-2] + match_rnj[len(match_rnj)-1]

print(good_match)
# remove strings that contain "formand" as it's not relevant information
good_match = [x for x in good_match if "formand" not in x.lower() if "\r" not in x.lower()]
print(good_match)
print(len(good_match))

for i in range(len(good_match)-1):
    name = good_match[i].lower()
    party_string = good_match[i+1]
    if name in data['Names']:
        print(name+ "found in "+ party_string.replace("Parti: ", ""))
        data.loc[data['Names'] == name, 'Party'] = party_string.replace("Parti: ", "")
        data.loc[data['Names'] == name, 'Region'] = 'nordjylland'


['Ulla Astman', 'Parti: Socialdemokratiet', 'Regionsrådsformand', 'Lone Sondrup', 'Parti: Venstre, Danmarks Liberale Parti', 'Næstformand', 'Lene Linnemann', 'Parti: SF - Socialistisk Folkeparti     ', 'Næstformand', 'Anne Mette Mortensen', 'Parti: Dansk Folkeparti', 'Anny Winther', 'Parti: Venstre, Danmarks Liberale Parti', 'Arne Nielsen', 'Parti: Socialdemokratiet', 'Bente Bang', 'Parti: Socialdemokratiet', 'Ejner Guldager Nielsen', 'Parti: Socialdemokratiet', 'Erik Harbo Larsen', 'Parti: Venstre, Danmarks Liberale Parti', 'Erik Høgh-Sørensen\r', 'Erik Høgh-Sørensen', 'Parti: Dansk Folkeparti', 'Gitte Lopdrup', 'Parti: Radikale Venstre', 'Hanne Korsgaard', 'Parti: Socialdemokratiet', 'Henrik Buchhave', 'Parti: Venstre, Danmarks Liberale Parti', 'Ib Madsen', 'Parti: Dansk Folkeparti', 'Jess V. Laursen', 'Parti: Venstre, Danmarks Liberale Parti', 'Jette Ramskov', 'Parti: Socialdemokratiet', 'Jørgen Hammer Sørensen', 'Parti: Dansk Folkeparti', 'Jørgen Hansen', 'Parti: Socialdemokratiet'

In [24]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,1,1,1
Dansk Folkeparti,13,13,13
Det Konservative Folkeparti,14,14,14
Enhedslisten,12,12,12
Inuit Ataqatigiit,1,1,1
Liberal Alliance,5,5,5
Radikale Venstre,15,15,15
Socialdemokratiet,42,42,42
Socialistisk Folkeparti,19,19,19


# Region Midtjylland

In [25]:
url = 'https://www.rm.dk/politik/regionsradsmedlemmer/'

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

content = soup.find("div", {"class": "text-content"})

unwanted = content.findAll("p")
for u in unwanted:
    u.clear()

# removing some introductioniary text
unwanted = content.find("h2")
unwanted.clear()
unwanted.find_next().clear()
clean_content = content.text.rstrip()

In [26]:
# removing unwanted brackets ()
pattern = r"\(.*?\)"
clean_content = re.sub(pattern, '', clean_content)


#print(clean_content) # left with this string

In [27]:
all_parties = content.findAll("h3")
for party in all_parties:
    #print(party.text)
    party_text = re.sub(pattern,'', party.text)
    people = party.find_next('ul')
    people_text = re.sub(pattern, "", people.text)
    for name in people_text.split('\n'):
        if name in data.values:
            print(name + "  in   " + party_text)
            data.loc[data['Names'] == name, 'Party'] = party_text.rstrip()
            data.loc[data['Names'] == name, 'Region'] = 'midtjylland'

Anders Kühnau  in   Socialdemokratiet 
Annette Roed  in   Socialdemokratiet 
Claus Kjeldsen  in   Socialdemokratiet 
Mette Valbjørn  in   Socialdemokratiet 
Hanne Roed  in   Radikale Venstre 
Nicolaj Bang  in   Det Konservative Folkeparti 
Jacob Isøe Klærke  in   Socialistisk Folkeparti 
Jakob Rixen  in   Liberal Alliance 
Lone Langballe  in   Dansk Folkeparti 
Ole Revsgaard Andersen  in   Dansk Folkeparti 
Arne Lægaard  in   Venstre 
Carsten Kissmeyer  in   Venstre 
Dorte West  in   Venstre 
Steen Jakobsen  in   Venstre 
Ulrich Fredberg  in   Venstre 
Else Kayser  in   Enhedslisten 
Rasmus Foged  in   Alternativet 


In [28]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,15,15,15
Det Konservative Folkeparti,15,15,15
Enhedslisten,13,13,13
Inuit Ataqatigiit,1,1,1
Liberal Alliance,6,6,6
Radikale Venstre,16,16,16
Socialdemokratiet,46,46,46
Socialistisk Folkeparti,19,19,19


In [29]:
data.Party.unique()

array(['Radikale Venstre', 'Venstre', 'Enhedslisten', 'UNKNOWN',
       'Det Konservative Folkeparti', 'Socialistisk Folkeparti',
       'Socialdemokratiet', 'Dansk Folkeparti', 'Liberal Alliance',
       'Alternativet', 'Inuit Ataqatigiit',
       'Uden for folketingsgrupperne'], dtype=object)

# Region syddammark

In [30]:
url = 'https://www.regionsyddanmark.dk/wm436558'
req = requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')
content = soup.find("div", {"class": "brodtekst"})

unwanted = content.findAll("p")
for u in unwanted:
    u.clear()


In [31]:
all_parties = content.findAll("h3")
i = 0
for party in all_parties:
        party_text = re.sub(pattern,'', party.text)
        people = party.find_next('ul')
        people_text = re.sub(pattern, "", people.text)
        for people in people_text.split('\n'):
            if people.strip() in data.values:
                print(people.strip() + "  in   " + party_text.strip())
                data.loc[data['Names'] == people.strip(), 'Party'] = party_text.strip()
                data.loc[data['Names'] == name, 'Region'] = 'syddanmark'

Poul-Erik Svendsen  in   Socialdemokratiet
Karsten Uno Petersen  in   Socialdemokratiet
Bente Gertz  in   Socialdemokratiet
Poul Fremmelev  in   Socialdemokratiet
Villy Søvndal  in   Socialistisk Folkeparti
Ida Damborg  in   Socialistisk Folkeparti
Karsten Fogde  in   Socialistisk Folkeparti
Thies Mathiasen  in   Dansk Folkeparti
Søren Rasmussen  in   Dansk Folkeparti
Stephanie Lose  in   Venstre
Tage Petersen  in   Venstre
Mette Bossen Linnet  in   Venstre
Kristian Nørgaard  in   Venstre
Preben Friis-Hauge  in   Venstre
Anja Lund  in   Venstre
Lars Mogensen  in   Enhedslisten


In [32]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,17,17,17
Det Konservative Folkeparti,15,15,15
Enhedslisten,14,14,14
Inuit Ataqatigiit,1,1,1
Liberal Alliance,6,6,6
Radikale Venstre,16,16,16
Socialdemokratiet,50,50,50
Socialistisk Folkeparti,20,20,20


# Region Sjælland

In [33]:
url = 'https://www.regionsjaelland.dk/politik/regionsraadet/regionsraadets-medlemmer/Sider/default.aspx'
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

content = soup.findAll('div',{'class': 'employeList'})
content = content[1:]
for name_content in content:
    name = name_content.a['title']
    spans = name_content.findAll('span')
    party = spans[1]

    party_text = party.text.split(',')[1]
    party_text = re.sub(pattern, '', party_text)
    if name.strip() in data.values:
        print(name.strip() + "  in   " + party_text.strip())
        data.loc[data['Names'] == name.strip(), 'Party'] = party_text.strip()
        data.loc[data['Names'] == name, 'Region'] = 'sjælland'


Jacob Jensen  in   Venstre
Claus Bakke  in   Venstre
Kirsten Devantier  in   Venstre
Lars Folmann  in   Dansk Folkeparti
Jan Hendeliowitz  in   Socialdemokraterne
Bruno Jerup  in   Enhedslisten
Annemarie Knigge  in   Socialdemokraterne
Camilla Hove Lund  in   Venstre
Susanne Lundvald  in   Socialdemokraterne
Evan Lynnerup  in   Venstre
Lene Madsen Milner  in   Venstre
Felex Pedersen  in   Socialdemokratiet
Kirsten Rask  in   Socialdemokraterne
Anne Møller Ronex  in   Radikale Venstre
Gitte Simoni  in   Dansk Folkeparti
Bodil Sø  in   Venstre
Lars Hoppe Søe  in   Radikale Venstre


In [34]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,19,19,19
Det Konservative Folkeparti,15,15,15
Enhedslisten,14,14,14
Inuit Ataqatigiit,1,1,1
Liberal Alliance,6,6,6
Radikale Venstre,18,18,18
Socialdemokraterne,4,4,4
Socialdemokratiet,51,51,51


# Region Hovedstaden

In [35]:
url = 'https://www.regionh.dk/politik/politiske-udvalg-og-fora/regionsraadet/Sider/Medlemmer_af_regionsraadet_2018-2021.aspx'

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
content = soup.findAll('h2', {'class': 'rh-Element-H2A'})

for party in content:
    print(party.text)
    party_text = re.sub(pattern, '', party.text)
    print(party_text.strip())
    print("")
    names = party.find_all_next('h3', {'rh-Element-H3A'})
    # bit weird website structure compared to others so differnet method is used
    # each party returns the names of politicians + those listed after it
    # we can update each name setting it to the party continously for each element
    for name in names:
        name_text = name.text
        name_text = name_text.strip()
        #print(name_text)
        if name_text in data.values:
                print(name_text + " in " + party_text.strip())
                data.loc[data['Names'] == name_text, 'Party'] = party_text.strip()
                data.loc[data['Names'] == name, 'Region'] = 'hovedstaden'


(A) Socialdemokratiet 
Socialdemokratiet

Özkan Kocak in Socialdemokratiet
Flemming Pless in Socialdemokratiet
Lars Gaardhøj in Socialdemokratiet
Maria Gudme in Socialdemokratiet
Bodil Kornbek in Socialdemokratiet
Stinus Lindgreen in Socialdemokratiet
Annette Randløv in Socialdemokratiet
Hans Toft in Socialdemokratiet
Jens Mandrup in Socialdemokratiet
Peter Westermann in Socialdemokratiet
Karoline Vind in Socialdemokratiet
Anne Ehrenreich in Socialdemokratiet
Randi Mondorf in Socialdemokratiet
Marianne Frederik in Socialdemokratiet
Annie Hagel in Socialdemokratiet



Stinus Lindgreen in 
Annette Randløv in 
Hans Toft in 
Jens Mandrup in 
Peter Westermann in 
Karoline Vind in 
Anne Ehrenreich in 
Randi Mondorf in 
Marianne Frederik in 
Annie Hagel in 
(B) Radikale Venstre
Radikale Venstre

Stinus Lindgreen in Radikale Venstre
Annette Randløv in Radikale Venstre
Hans Toft in Radikale Venstre
Jens Mandrup in Radikale Venstre
Peter Westermann in Radikale Venstre
Karoline Vind in Radikale V

In [36]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,19,19,19
Det Konservative Folkeparti,16,16,16
Enhedslisten,14,14,14
Enhedslisten - De Rød-Grønne,3,3,3
Inuit Ataqatigiit,1,1,1
Liberal Alliance,6,6,6
Radikale Venstre,20,20,20
SF - Socialistisk Folkeparti,3,3,3


In [37]:
# get snapshot of who's missing
data.loc[data['Party'] == "UNKNOWN"]

,Names,Twittername,Party,Region
6,Uffe Elbaek,uffeelbaek,UNKNOWN,UNKNOWN
10,Sofie CarstenNielsen,sofiecn,UNKNOWN,UNKNOWN
12,Regeringen,regeringDK,UNKNOWN,UNKNOWN
13,Brian Holm,brianholm1962,UNKNOWN,UNKNOWN
14,Simon Emil Ammitzbøll-Bille,ammitzbollbille,UNKNOWN,UNKNOWN
...,...,...,...,...
645,Christina Kjærsgaard,ChristinaKjrsg2,UNKNOWN,UNKNOWN
646,Molle lykke nielsen,LykkeMolle,UNKNOWN,UNKNOWN
647,Vibeke Gamst,vibeke_gamst,UNKNOWN,UNKNOWN
648,Pia Moldt,piamoldt,UNKNOWN,UNKNOWN


## Finally from ft.dk

In [38]:
### nordjyllands storkreds
def getFromFt(url: str, region: str):
    req=requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    memberInfos = soup.findAll('tr')
    # skip table header
    memberInfos = memberInfos[1:]
    print("Found " + str(len(memberInfos)))
    for member in memberInfos:
        # format: '\n\n -> first_name -> last_name -> party
        dataList = [x.text for x in member.findAll('td')]
        try:
            _, firstName, lastName, party, _, _ = dataList
            name = firstName + " " + lastName
            if name in data.values:
                print("In twitter csv: " + name + " " + party)
                print("Old party: " + data.loc[data['Names']==name]['Party'])
                data.loc[data['Names'] == name, 'Party'] = party
                data.loc[data['Names'] == name, 'Region'] = region

        except:
            print("failed for")
            print(dataList)



url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti=%7bD68D9A59-8A4B-426D-AD9C-9B32FE3CE071%7d&page=1&sortedBy=&pageSize=200'
getFromFt(url, 'nordjylland')
data.groupby('Party').count()

Found 19
In twitter csv: Lisbeth Bech-Nielsen Socialistisk Folkeparti
59    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Marie Bjerre Venstre
223    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Orla Hav Socialdemokratiet
220    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Anne Honoré Østergaard Venstre
314    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Peder Hvelplund Enhedslisten
183    Old party: Enhedslisten
Name: Party, dtype: object
In twitter csv: Marianne Jelved Radikale Venstre
97    Old party: Radikale Venstre
Name: Party, dtype: object
In twitter csv: Simon Kollerup Socialdemokratiet
68    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Per Larsen Det Konservative Folkeparti
248    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Karsten Lauritzen Venstre
33    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Bjarne Laustsen Socialdemokratiet
328   

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,19,19,19
Det Konservative Folkeparti,17,17,17
Enhedslisten,14,14,14
Enhedslisten - De Rød-Grønne,3,3,3
Inuit Ataqatigiit,1,1,1
Liberal Alliance,6,6,6
Radikale Venstre,20,20,20
SF - Socialistisk Folkeparti,3,3,3


In [39]:
## Vestjyllandsstokkreds + øst
# trying same code

url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={97A50F7E-B9D0-4D91-924F-AB20235DB4FD}'
getFromFt(url, 'midtjylland')
data.groupby('Party').count()

# østjylland
url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={E3910753-B21F-4E97-AD2B-67A9A3C15B62}'
getFromFt(url, 'midtjylland')
data.groupby('Party').count()

Found 16
In twitter csv: Thomas Danielsen Venstre
129    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Dennis Flydtkjær Dansk Folkeparti
136    Old party: Dansk Folkeparti
Name: Party, dtype: object
In twitter csv: Mogens Jensen Socialdemokratiet
34    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Kristian Jensen Venstre
5    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Carsten Kissmeyer Venstre
311    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Annette Lind Socialdemokratiet
192    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Signe Munk Socialistisk Folkeparti
120    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Anne Paulin Socialdemokratiet
196    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Søren Pape Poulsen Det Konservative Folkeparti
7    Old party: Det Konservative Folkeparti
Name: Party, dtype: object
In twitter csv: Andreas Steenber

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,20,20,20
Det Konservative Folkeparti,18,18,18
Enhedslisten,14,14,14
Enhedslisten - De Rød-Grønne,3,3,3
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Nye Borgerlige,1,1,1
Radikale Venstre,22,22,22


In [40]:
# syddanmark
url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={7D7567F4-3425-42A1-A035-2FB9880A5F81}'
getFromFt(url, 'syddanmark')
data.groupby('Party').count()


url ='https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={B3A26626-55E1-4BE2-8503-E4027904E25A}'
getFromFt(url, 'syddanmark')
data.groupby('Party').count()

Found 21
In twitter csv: Karina Adsbøl Dansk Folkeparti
148    Old party: Dansk Folkeparti
Name: Party, dtype: object
In twitter csv: Henrik Dahl Liberal Alliance
81    Old party: Liberal Alliance
Name: Party, dtype: object
In twitter csv: Kristian Thulesen Dahl Dansk Folkeparti
37    Old party: Dansk Folkeparti
Name: Party, dtype: object
In twitter csv: Benny Engelbrecht Socialdemokratiet
29    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Niels Flemming Hansen Det Konservative Folkeparti
304    Old party: Det Konservative Folkeparti
Name: Party, dtype: object
In twitter csv: Eva Kjer Hansen Venstre
127    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Marie Krarup Dansk Folkeparti
107    Old party: Dansk Folkeparti
Name: Party, dtype: object
In twitter csv: Anders Kronborg Socialdemokratiet
229    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Anni Matthiesen Venstre
142    Old party: UNKNOWN
Name: Party, dtype: object
In twitte

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,20,20,20
Det Konservative Folkeparti,18,18,18
Enhedslisten,14,14,14
Enhedslisten - De Rød-Grønne,3,3,3
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Nye Borgerlige,2,2,2
Radikale Venstre,22,22,22


In [41]:
url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti=%7b79B9CCF4-4793-4835-8E68-6770AA11A85D%7d&page=1&sortedBy=&pageSize=50'
getFromFt(url, 'sjælland')
data.groupby('Party').count()

Found 29
In twitter csv: Anne Valentina Berthelsen Socialistisk Folkeparti
208    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Liselott Blixt Dansk Folkeparti
82    Old party: Dansk Folkeparti
Name: Party, dtype: object
In twitter csv: Astrid Carøe Socialistisk Folkeparti
205    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Morten Dahlin Venstre
117    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Louise Schack Elholm Venstre
131    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Søren Espersen Dansk Folkeparti
55    Old party: Dansk Folkeparti
Name: Party, dtype: object
In twitter csv: Eva Flyvholm Enhedslisten
144    Old party: Enhedslisten
Name: Party, dtype: object
In twitter csv: Mette Gjerskov Socialdemokratiet
51    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Magnus Heunicke Socialdemokratiet
9    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Bertel Haarder Venstr

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,20,20,20
Det Konservative Folkeparti,18,18,18
Enhedslisten,14,14,14
Enhedslisten - De Rød-Grønne,3,3,3
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Nye Borgerlige,3,3,3
Radikale Venstre,23,23,23


In [42]:
# Region hovedstadan
url1 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={4C1C732F-4F3C-408E-A009-C96E6934B29C}'
url2 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={3F642D1B-1820-48F1-A288-AA1FE9079640}'
url3 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={6A4F6A85-C5C1-4656-BD9A-7831094A5A81}'
url4 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={E76B8CF6-E8A7-4B29-9813-350C49E36238}'

getFromFt(url1, 'hovedstadan')
getFromFt(url2, 'hovedstadan')
getFromFt(url3, 'hovedstadan')
getFromFt(url4, 'hovedstadan')

data.groupby('Party').count()

Found 20
In twitter csv: Tommy Ahlers Venstre
24    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Simon Emil Ammitzbøll-Bille Uden for folketingsgrupperne
14    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Ida Auken Radikale Venstre
4    Old party: Radikale Venstre
Name: Party, dtype: object
In twitter csv: Pia Olsen Dyhr Socialistisk Folkeparti
8    Old party: Socialistisk Folkeparti
Name: Party, dtype: object
In twitter csv: Martin Geertsen Venstre
213    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Peter Hummelgaard Socialdemokratiet
31    Old party: Socialdemokratiet
Name: Party, dtype: object
In twitter csv: Jan E. Jørgensen Venstre
32    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Rosa Lund Enhedslisten
58    Old party: Enhedslisten
Name: Party, dtype: object
In twitter csv: Rune Lund Enhedslisten
143    Old party: Enhedslisten
Name: Party, dtype: object
In twitter csv: Samira Nawa Radikale Venstre
104    Old par

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,20,20,20
Det Konservative Folkeparti,19,19,19
Enhedslisten,16,16,16
Enhedslisten - De Rød-Grønne,3,3,3
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Nye Borgerlige,4,4,4
Radikale Venstre,23,23,23


### Analyzing why there are still so many unknowns

In [43]:
# unknowns = data.loc[data['Party'] == "UNKNOWN"]
#
# for row in unknowns['Names']:
#     print(row)

Looks like there are several reasons, some names have emojis. Some have misplaced whitespaces, sometimes more
sometimes 0. There are some names in the list that are entities and not people - such as AMPJ or various ministeries.
There can also be trouble finding politicians because of the capitalization (easy fix) and laslty,
some middle names are abbreviated and some aren't; making it harder to know and find the specific politician.


## Duplicates removal  =) 

In [44]:
#SF - Socialistisk Folkeparti = Socialistisk Folkeparti
# Venstre = Venstre, Danmarks Liberale Parti
# Socialdemokratiet = Socialdemokraterne
party_replacements = {
        'Venstre, Danmarks Liberale Parti': 'Venstre',
        'Socialdemokraterne': 'Socialdemokratiet',
        'SF - Socialistisk Folkeparti': 'Socialistisk Folkeparti'
}

to_replace = ['Venstre, Danmarks Liberale Parti', 'Socialdemokraterne', 'SF - Socialistisk Folkeparti','SF - Socialistisk Folkeparti']

data["Party"] = data.apply(lambda x: party_replacements[x["Party"]] if x["Party"] in to_replace else x["Party"], axis=1)

data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,2,2,2
Dansk Folkeparti,20,20,20
Det Konservative Folkeparti,19,19,19
Enhedslisten,16,16,16
Enhedslisten - De Rød-Grønne,3,3,3
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Nye Borgerlige,4,4,4
Radikale Venstre,23,23,23


In [45]:
data.loc[data['Party'] == "UNKNOWN"]

,Names,Twittername,Party,Region
6,Uffe Elbaek,uffeelbaek,UNKNOWN,UNKNOWN
10,Sofie CarstenNielsen,sofiecn,UNKNOWN,UNKNOWN
12,Regeringen,regeringDK,UNKNOWN,UNKNOWN
13,Brian Holm,brianholm1962,UNKNOWN,UNKNOWN
17,Denmark MFA 🇩🇰,DanishMFA,UNKNOWN,UNKNOWN
...,...,...,...,...
645,Christina Kjærsgaard,ChristinaKjrsg2,UNKNOWN,UNKNOWN
646,Molle lykke nielsen,LykkeMolle,UNKNOWN,UNKNOWN
647,Vibeke Gamst,vibeke_gamst,UNKNOWN,UNKNOWN
648,Pia Moldt,piamoldt,UNKNOWN,UNKNOWN


### top 5 kommuner

In [46]:
url = "https://www.kk.dk/indhold/medlemsoversigt-1"




req=requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
party_panel = soup.findAll('div',{'class': 'reg-inner-regions'})[0]
parties_inner = party_panel.findChildren('div', recursive=False)
region = "hovedstadan"
for party_p in parties_inner:
    party = party_p.h2.text
    party = party.strip()
    names = party_p.findAll("a", {"class": "node-link"})
    for name in names:
        name = name.text
        name = re.sub(r" ?\([^)]+\)", "", name) # remove brackets
        name = re.sub(r",.+", "", name) # remove title
        
        if name in data.values:
            print("In twitter csv: " + name + " " + party)
            print("Old party: " + data.loc[data['Names']==name]['Party'])
            data.loc[data['Names'] == name, 'Party'] = party
            data.loc[data['Names'] == name, 'Region'] = region




In twitter csv: Jesper Christensen Socialdemokratiet
175    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Jonas Bjørn Jensen Socialdemokratiet
226    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Laura Rosenvinge Socialdemokratiet
376    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Marcus Vesterager Socialdemokratiet
306    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Niels E. Bjerrum Socialdemokratiet
286    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Simon Simonsen Socialdemokratiet
303    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Simon Strange Socialdemokratiet
387    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Sofie Seidenfaden Socialdemokratiet
312    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Susan Hedlund Socialdemokratiet
611    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Trine Madsen Socialdemokratiet
388    Old party: U

In [47]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,4,4,4
Dansk Folkeparti,21,21,21
Det Konservative Folkeparti,19,19,19
Enhedslisten,20,20,20
Enhedslisten - De Rød-Grønne,3,3,3
"Frie Grønne, Danmarks Nye Venstrefløjsparti",1,1,1
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Løsgænger,1,1,1


In [48]:
url = "https://www.aarhus.dk/demokrati/politik/byraadet/byraadets-medlemmer/"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
# århus
region = 'midtjylland'
articles = soup.findAll('article')

for article in articles:
    name = article.h2.text.strip()
    name = re.sub(r",.+", "", name) # remove title
    #print(name)
    party = article.p.text.split('(')[0].strip()
    #print(party)
    if name in data.values:
        print("In twitter csv: " + name + " " + party)
        print("Old party: " + data.loc[data['Names']==name]['Party'])
        data.loc[data['Names'] == name, 'Party'] = party
        data.loc[data['Names'] == name, 'Region'] = region
        
    

In twitter csv: Jacob Bundsgaard Socialdemokratiet
92    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Thomas Medom Socialistisk Folkeparti
166    Old party: Socialistisk Folkeparti
Name: Party, dtype: object
In twitter csv: Bünyamin Simsek Venstre
247    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Rabih Azad-Ahmad Radikale Venstre
464    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Anders Winnerskjold Socialdemokratiet
384    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Gert Bjerregaard Venstre
351    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Hans Skou Venstre
550    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Heidi Bank Venstre
281    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Jan Ravn Christensen Socialistisk Folkeparti
386    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Lene Horsbøl Venstre
288    Old party: UNKNOWN
Name: Party, dtype: object

In [49]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,4,4,4
Dansk Folkeparti,21,21,21
Det Konservative Folkeparti,19,19,19
Enhedslisten,20,20,20
Enhedslisten - De Rød-Grønne,3,3,3
"Frie Grønne, Danmarks Nye Venstrefløjsparti",1,1,1
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Løsgænger,1,1,1


In [50]:
# region nordylland/aalborg
region = "nordjylland"
url = 'https://www.aalborg.dk/politik/aalborg-byraad/politikere'
req = requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')
party_panels = soup.findAll('div',{'class': 'grid_4'})

for party_p in party_panels:
    party = party_p.h2.text.strip()
    print("")

    print(party)
    print("")
    names = party_p.findAll('a')
    for _name in names:
        name = _name.text.strip()
        if name in data.values:
            print("In twitter csv: " + name + " " + party)
            print("Old party: " + data.loc[data['Names']==name]['Party'])
            data.loc[data['Names'] == name, 'Party'] = party
            data.loc[data['Names'] == name, 'Region'] = region



Socialdemokratiet

In twitter csv: Lasse Frimand Jensen Socialdemokratiet
273    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Lene Krabbe Dahl Socialdemokratiet
500    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Søren Kusk Socialdemokratiet
427    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Rose Sloth Hansen Socialdemokratiet
324    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Lisbeth Lauritsen Socialdemokratiet
284    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Anna Kirsten Olesen Socialdemokratiet
582    Old party: UNKNOWN
Name: Party, dtype: object

Venstre

In twitter csv: Tina French Nielsen Venstre
417    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Mads Duedahl Venstre
404    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Maja Torp Venstre
416    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Carsten Kristensen Venstre
509    Old party: UNKNOWN


In [51]:
# odense region syddanmark
url = 'https://www.odense.dk/politik/odense-byraad/byraadets-medlemmer'
req = requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')
region = 'syddanmark'
container = soup.find("div", {"class": {"richtext"}})
#print(container)
# socdem is special

ps = container.findAll('p')
party = "Socialdemokratiet"

for politician in ps[1].findAll('a'):
    politician = politician.text.strip()
    if politician in data.values:
        print(politician)
        data.loc[data['Names'] == name, 'Party'] = party
        data.loc[data['Names'] == name, 'Region'] = region
        
# other parties
parties = container.findAll('h2')
for _party in parties:
    party = _party.text.strip()
    party = re.sub(r" ?\([^)]+\)", "", party) # remove brackets

    for pol in _party.findNext('p').findAll('a'):
        politician = pol.text.strip()
        #print(politician + " in " + party)
        if politician in data.values:
            print("In twitter csv: " + name + " " + party)
            print("Old party: " + data.loc[data['Names']==politician]['Party'])
            data.loc[data['Names'] == politician, 'Party'] = party
            data.loc[data['Names'] == politician, 'Region'] = region

Hjalte Daniel Hansen
Kasper Ejlertsen
Maria Brumvig
Michael Eskamp Witek
Tim Vermund
In twitter csv: Ole Risager Det Konservative Folkeparti
294    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Ole Risager Det Konservative Folkeparti
491    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Ole Risager Det Konservative Folkeparti
451    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Ole Risager Socialistisk Folkeparti
298    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Ole Risager Dansk Folkeparti
219    Old party: Dansk Folkeparti
Name: Party, dtype: object
In twitter csv: Ole Risager Venstre, Danmarks Liberale Parti
260    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Ole Risager Venstre, Danmarks Liberale Parti
353    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Ole Risager Venstre, Danmarks Liberale Parti
283    Old party: Venstre
Name: Party, dtype: object
In twitter csv: Ole Risager Venstr

In [52]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,4,4,4
Dansk Folkeparti,21,21,21
Det Konservative Folkeparti,22,22,22
Det Radikale Venstre,1,1,1
Enhedslisten,20,20,20
Enhedslisten - De Rød-Grønne,3,3,3
"Frie Grønne, Danmarks Nye Venstrefløjsparti",1,1,1
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7


In [ ]:
## venstre hjemmeside

https://www.venstre.dk/personer/kommuner

## Duplicates

In [53]:
#SF - Socialistisk Folkeparti = Socialistisk Folkeparti
# Venstre = Venstre, Danmarks Liberale Parti
# Socialdemokratiet = Socialdemokraterne
party_replacements = {
        'Venstre, Danmarks Liberale Parti': 'Venstre',
        'Socialdemokraterne': 'Socialdemokratiet',
        'SF - Socialistisk Folkeparti': 'Socialistisk Folkeparti',
        'Enhedslisten - De Rød-Grønne': 'Enhedslisten',
        'Løsgænger': 'Uden for folketingsgrupperne',
        'Frie Grønne, Danmarks Nye Venstrefløjsparti': 'Uden for folketingsgrupperne'
}

to_replace = ['Venstre, Danmarks Liberale Parti', 'Socialdemokraterne', 'SF - Socialistisk Folkeparti','SF - Socialistisk Folkeparti','Enhedslisten - De Rød-Grønne', 'Løsgænger', 'Frie Grønne, Danmarks Nye Venstrefløjsparti']

data["Party"] = data.apply(lambda x: party_replacements[x["Party"]] if x["Party"] in to_replace else x["Party"], axis=1)

data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,4,4,4
Dansk Folkeparti,21,21,21
Det Konservative Folkeparti,22,22,22
Det Radikale Venstre,1,1,1
Enhedslisten,23,23,23
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Nye Borgerlige,4,4,4
Radikale Venstre,25,25,25


In [54]:
unknowns = data.loc[data['Party'] == "UNKNOWN"]
party = 'Government'
region = 'government'
for x in unknowns.iterrows():
    politician = x[1][0]
    if re.match(r'.+ministeriet', politician):
        if politician in data.values:
            print("In twitter csv: " + politician)
            print("Old party: " + data.loc[data['Names']==politician]['Party'])
            data.loc[data['Names'] == politician, 'Party'] = party
            data.loc[data['Names'] == politician, 'Region'] = region


In twitter csv: Uddannelses- og Forskningsministeriet
54    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Finansministeriet
65    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Erhvervsministeriet
72    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Sundheds- og Ældreministeriet
73    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Miljøministeriet
79    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Forsvarsministeriet/Danish MoD
98    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Kulturministeriet
99    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Klimaministeriet
102    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Børne- og Undervisningsministeriet
106    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Skatteministeriet
141    Old party: UNKNOWN
Name: Party, dtype: object
In twitter csv: Transport- og Boligministeriet
152    Old party: UNKNOWN
Na

In [55]:
#Some special cases 
data.loc[data['Names'] == "Regeringen", 'Party'] = 'Government'
data.loc[data['Names'] == "Regeringen", 'Region'] = 'Government'

print(data.loc[data['Names']=="Regeringen"]['Party'])



12    Government
Name: Party, dtype: object


In [56]:
data.groupby('Party').count()

,Names,Twittername,Region
Party,,,
Alternativet,4,4,4
Dansk Folkeparti,21,21,21
Det Konservative Folkeparti,22,22,22
Det Radikale Venstre,1,1,1
Enhedslisten,23,23,23
Government,14,14,14
Inuit Ataqatigiit,1,1,1
Liberal Alliance,7,7,7
Nye Borgerlige,4,4,4


In [57]:
# write to csv
data.to_csv('danish_politkere_twitter2.csv',index= False)